# Tensorflow 2.0 

Check Tensorflow installation

In [1]:
import tensorflow as tf
print(tf.version)

<module 'tensorflow._api.v2.version' from 'c:\\Users\\phank\\AppData\\Local\\Programs\\Python\\Python310\\lib\\site-packages\\tensorflow\\_api\\v2\\version\\__init__.py'>


In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import clear_output
from six.moves import urllib

import tensorflow.compat.v2.feature_column as fc

import tensorflow as tf

### Creating Tensor
A tensor is a generalization of vectors and matrices to potentially higher dimensions. Internally, TensorFlow represents tensors as n-dimensional arrays of base datatypes.

Creating Scalar (RANK 0 tensor)

In [ ]:
tensor_string = tf.Variable("This is a tensor string", tf.string)
tensor_int = tf.Variable(420, tf.int16)
tensor_float = tf.Variable(69.420, tf.float32)

Creating Vectors/Matrices (RANK >0 tensor)

In [ ]:
rank1_tensor = tf.Variable([69,420], tf.int16)
rank2_tensor = tf.Variable([[49,420],[9,11]], tf.int16)

In [ ]:
tf.rank(rank2_tensor)

In [ ]:
rank2_tensor.shape

Changing shape of a tensor

In [ ]:
tensor1 = tf.ones([2,2,3])
print(tensor1)
tensor2 = tf.reshape(tensor1,[2,3,2])
print(tensor2)
tensor3 = tf.reshape(tensor1,[12])
print(tensor3)

# TensorFlow Core Learning Algorithms

## Linear Regression
Linear regression is one of the most basic forms of machine learning and is used to predict numeric values.

Use when the datapoint is correlated linearly

Example of Data for linear regression

In [ ]:
x = [1, 2, 2.5, 3, 4]
y = [1, 4, 7, 9, 15]
plt.plot(x, y, 'ro')
plt.axis([0, 6, 0, 20])

Line of best fit for this example.

In [ ]:
plt.plot(x, y, 'ro')
plt.axis([0, 6, 0, 20])
plt.plot(np.unique(x), np.poly1d(np.polyfit(x, y, 1))(np.unique(x)))
plt.show()

Creating Model to predicting the Line of best fit

Load Dataset

- Titanic dataset contain list and attribute of people boarding Titanic at the time of the crash

In [ ]:
dftrain = pd.read_csv('https://storage.googleapis.com/tf-datasets/titanic/train.csv') # training data
dfeval = pd.read_csv('https://storage.googleapis.com/tf-datasets/titanic/eval.csv') # testing data
y_train = dftrain.pop('survived')
y_eval = dfeval.pop('survived')

In [ ]:
dftrain.pop('n_siblings_spouses')
dftrain.pop('fare')
dftrain.head()

In [ ]:
dftrain.describe()

In [ ]:
dftrain.age.hist(bins=20)

In [ ]:
dftrain.sex.value_counts().plot(kind="barh")

In [ ]:
pd.concat([dftrain, y_train], axis=1).groupby('sex').survived.mean().plot(kind='barh').set_xlabel('% survive')

Encoding categorical and numeric data into tensor

In [ ]:
CATEGORICAL_COLUMNS  = ['sex','parch','class','deck','embark_town','alone']
NUMERICAL_COLUMNS = ['age']

feature_columns = []

for feature_name in CATEGORICAL_COLUMNS:
    vocabulary = dftrain[feature_name].unique()
    feature_columns.append(tf.feature_column.categorical_column_with_vocabulary_list(feature_name, vocabulary))

for feature_name in NUMERICAL_COLUMNS:
    feature_columns.append(tf.feature_column.numeric_column(feature_name, dtype=tf.float32))

feature_columns

### Training Process
Load small amount of data (batches) into the model using input function

In [ ]:
def make_input_fn(data_df, label_df, num_epochs=16, shuffle=True, batch_size=32):
  def input_function():  # inner function, this will be returned
    ds = tf.data.Dataset.from_tensor_slices((dict(data_df), label_df))  # create tf.data.Dataset object with data and its label
    if shuffle:
      ds = ds.shuffle(1000)  # randomize order of data
    ds = ds.batch(batch_size).repeat(num_epochs)  # split dataset into batches of 32 and repeat process for number of epochs
    return ds  # return a batch of the dataset
  return input_function  # return a function object for use

train_input_fn = make_input_fn(dftrain, y_train)  # here we will call the input_function that was returned to us to get a dataset object we can feed to the model
eval_input_fn = make_input_fn(dfeval, y_eval, num_epochs=1, shuffle=False)

### Creating a model

In [ ]:
linear_est = tf.estimator.LinearClassifier(feature_columns=feature_columns)

### Training a model

In [ ]:
linear_est.train(train_input_fn)  # train
#result = linear_est.evaluate(eval_input_fn)  # get model metrics/stats by testing on tetsing data
#clear_output()  # clears consoke output
#print(result)  # the result variable is simply a dict of stats about our model

In [ ]:
result = list(linear_est.predict(eval_input_fn))
correct = 0
incorrect = 0

for i in range(len(result)) :
    print(y_eval.loc[i])
    print(result[i]["probabilities"][1])
    if y_eval.loc[i] == round(result[i]["probabilities"][1]) :
        correct += 1
    else :
        incorrect += 1

accuracy = (correct / (correct + incorrect)) * 100
print("CORRECT PREDICTION : " + str(correct) + '/' + str(correct+incorrect))
print("INCORRECT PREDICTION : " + str(incorrect) + '/' + str(correct+incorrect))
print("ACCURACY : " + str(accuracy) + "%")

## Classification

- Dataset is Iris which is the daataset containing a different species of flower

- load dataset

In [9]:
CSV_COLUMN_NAMES = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth', 'Species']
SPECIES = ['Setosa', 'Versicolor', 'Virginica']
# Lets define some constants to help us later on
train_path = tf.keras.utils.get_file(
    "iris_training.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv")
test_path = tf.keras.utils.get_file(
    "iris_test.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_test.csv")

train = pd.read_csv(train_path, names=CSV_COLUMN_NAMES, header=0)
test = pd.read_csv(test_path, names=CSV_COLUMN_NAMES, header=0)
# Here we use keras (a module inside of TensorFlow) to grab our datasets and read them into a pandas dataframe

In [10]:
train.head()

,SepalLength,SepalWidth,PetalLength,PetalWidth,Species
0,6.4,2.8,5.6,2.2,2
1,5.0,2.3,3.3,1.0,1
2,4.9,2.5,4.5,1.7,2
3,4.9,3.1,1.5,0.1,0
4,5.7,3.8,1.7,0.3,0


In [11]:
train_y = train.pop('Species')
test_y = test.pop('Species')

In [12]:
train_y.head()
train.shape

(120, 4)

Creating input function

In [13]:
def input_fn(features, labels, training=True, batch_size=256):
    # Convert the inputs to a Dataset.
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))

    # Shuffle and repeat if you are in training mode.
    if training:
        dataset = dataset.shuffle(1000).repeat()
    
    return dataset.batch(batch_size)


In [14]:
# Feature columns describe how to use the input.
my_feature_columns = []
for key in train.keys():
    my_feature_columns.append(tf.feature_column.numeric_column(key=key))
print(my_feature_columns)

[NumericColumn(key='SepalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='SepalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PetalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PetalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]


In [15]:
my_feature_columns

[NumericColumn(key='SepalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='SepalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='PetalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='PetalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]

### Building Model
- DNNClassifier

In [16]:
# Build a DNN with 2 hidden layers with 30 and 10 hidden nodes each.
classifier = tf.estimator.DNNClassifier(
    feature_columns=my_feature_columns,
    # Two hidden layers of 30 and 10 nodes respectively.
    hidden_units=[50, 30],
    # The model must choose between 3 classes.
    n_classes=3)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\phank\\AppData\\Local\\Temp\\tmp4_xl7co4', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


Training a model

In [17]:
classifier.train(
    input_fn=lambda: input_fn(train, train_y, training=True),
    steps=5000)
# We include a lambda to avoid creating an inner function previously

INFO:tensorflow:Calling model_fn.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 0...
INFO:tensorflow:Saving checkpoints for 0 into C:\Users\phank\AppData\Local\Temp\tmp4_xl7co4\model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 0...
INFO:tensorflow:loss = 1.2308007, step = 0
INFO:tensorflow:global_step/sec: 175.605
INFO:tensorflow:loss = 0.94507265, step = 100 (0.571 sec)
INFO:tensorflow:global_step/sec: 207.853
INFO:tensorflow:loss = 0.84087837, step = 200 (0.481 sec)
INFO:tensorflow:global_step/sec: 204.035
INFO:tensorflow:loss = 0.78121346, step = 300 (0.490 sec)
INFO:tensorflow:global_step/sec: 208.721
INFO:tensor

Evaluating modeel

In [18]:
eval_result = classifier.evaluate(
    input_fn=lambda: input_fn(test, test_y, training=False))

print('\nTest set accuracy: {accuracy:0.3f}\n'.format(**eval_result))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2023-03-18T14:07:17
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\phank\AppData\Local\Temp\tmp4_xl7co4\model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 1.84942s
INFO:tensorflow:Finished evaluation at 2023-03-18-14:07:18
INFO:tensorflow:Saving dict for global step 5000: accuracy = 0.96666664, average_loss = 0.2567342, global_step = 5000, loss = 0.2567342
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 5000: C:\Users\phank\AppData\Local\Temp\tmp4_xl7co4\model.ckpt-5000

Test set accuracy: 0.967



Predicting using trained model

In [21]:
input_feature_columns = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth']
predict = {}

def usr_input_fn(features, batch_size=256):
    return tf.data.Dataset.from_tensor_slices(dict(features)).batch(batch_size)

print("user input")
for feature in input_feature_columns :
    val = input(feature + ' >> ')
    predict[feature] = [float(val)]

print('user input object : ' + str(predict))
predictions = classifier.predict(input_fn=lambda: usr_input_fn(predict))

for pred_dict in predictions:
    class_id = pred_dict['class_ids'][0]
    prob = pred_dict['probabilities'][class_id]
    print('Prediction is "{}" ({:.1f}%)'.format(SPECIES[class_id], 100*prob))


user input
user input object : {'SepalLength': [5.0], 'SepalWidth': [2.3], 'PetalLength': [3.3], 'PetalWidth': [1.0]}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\phank\AppData\Local\Temp\tmp4_xl7co4\model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Prediction is "Versicolor" (75.4%)


### Hidden Markov Model

finite set of state associated with a probability distribution useful for predicting future event based on past event

States : Could be anything

Observation : Each state will have a particular outcome or observation associated with it

Transitions : Each state will have a probability to transition to a different state

In [2]:
import tensorflow_probability as tfp
import tensorflow as tf

Example Data : Weather model
1. Cold days are encoded by a 0 and hot days are encoded by a 1.
2. The first day in our sequence has an 80% chance of being cold.
3. A cold day has a 30% chance of being followed by a hot day.
4. A hot day has a 20% chance of being followed by a cold day.
5. On each day the temperature is
 normally distributed with mean and standard deviation 0 and 5 on
 a cold day and mean and standard deviation 15 and 10 on a hot day.

In [3]:
tfd = tfp.distributions
initial_distribution = tfd.Categorical(probs=[0.2, 0.8]) 
transition_distribution = tfd.Categorical(probs=[[0.5, 0.5],
                                                 [0.2, 0.8]])
observation_distribution = tfd.Normal(loc=[0., 15.], scale=[5., 10.])